# Flavor Frontier Analysts 

Import Dependencies

In [1]:
import pandas as pd
from pymongo import MongoClient
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import json
import webbrowser
import os


Initialize Flask App and Set Up CORS

In [2]:
app = Flask(__name__)
CORS(app)

Connect to MongoDB and Create a Database and Collection

In [54]:
client = MongoClient(port=27017)
db = client['yelp_database']
collection = db['businesses']

Import the Yelp data .json 

In [55]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)

Covnert the categories field to string and then filter for results that include the string "Restaurants" in the 'categories' column.

In [57]:
df['categories'] = df['categories'].astype(str)
df_restaurants = df[df['categories'].str.contains('Restaurants', case=False, na=False)]

Create two lists from strings in the 'categories' column. One list of categories that are strictly restaurants, and another for food related categories. 
These lists will be passed to serve as the user input options as well as the main query variable.

In [63]:
strict_restaurant_categories = [
    "Restaurants", "American (Traditional)", "American (New)", "Italian", 
    "Mexican", "Chinese", "Japanese", "Thai", "Vietnamese", "Indian", "Korean", 
    "Mediterranean", "Greek", "French", "Spanish", "German", "Irish", "British", 
    "Seafood", "Steakhouses", "Pizza", "Burgers", "Sushi Bars", 
    "Barbeque", "Asian Fusion", "Diners", "Breakfast & Brunch", 
    "Buffets", "Comfort Food", "Soul Food", "Southern", 
    "Tex-Mex", "Cajun/Creole", "Latin American", "Middle Eastern", "African", 
    "Caribbean", "Brazilian", "Peruvian", "Cuban", "Tapas Bars", "Gastropubs",
    "Izakaya", "Ramen", "Poke", "Hot Pot", "Dim Sum", "Fondue",
    "Fish & Chips", "Bistros", "Brasseries",
    "Donairs", "Kebab", "Falafel", "Tacos", "Cheesesteaks"
]


food_related_categories = [
    "Food", "Sandwiches", "Cafes", "Fast Food", "Pubs", 
    "Halal", "Kosher", "Vegan", "Vegetarian",
    "Gluten-Free", "Organic", "Farm-to-table", "Food Trucks", "Food Stands",
    "Creperies", "Delis", "Noodles", "Soup", "Salad", "Chicken Wings",
    "Chicken Shop", "Hot Dogs", "Waffles", "Pancakes", "Donuts", "Bagels",
    "Food", "Specialty Food", "Caterers", "Bakeries", "Desserts", "Juice Bars & Smoothies",
    "Ice Cream & Frozen Yogurt", "Gelato", "Coffee & Tea"
]

In [64]:
 # Select relevant columns
df_restaurants_clean = df_restaurants[['business_id', 'name', 'city', 'state', 'latitude', 'longitude', 'categories', 'stars']]
df_restaurants_clean.head()

,business_id,name,city,state,latitude,longitude,categories,stars
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",4.0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",2.0
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,"Pubs, Restaurants, Italian, Bars, American (Tr...",3.0
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1.5
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,27.955269,-82.456320,"Vietnamese, Food, Restaurants, Food Trucks",4.0


In [65]:
# collection.delete_many({})

In [66]:
# Connect to MongoDB
if collection.count_documents({}) == 0:
    records = df_restaurants_clean.to_dict('records')
    collection.insert_many(records)
    print("Data loaded into MongoDB successfully.")
else:
    print("Data already exists in MongoDB.")

Data already exists in MongoDB.


In [67]:
# Setup flask api routes
@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify({
        'strict_restaurant_categories': strict_restaurant_categories
    })


In [68]:

@app.route('/query_restaurants/<category>', methods=['GET'])
def query_restaurants(category):
    query = {"categories": {'$regex': category, '$options': 'i'}}

    # Get results from MongoDB
    results = list(collection.find(query))

    # Convert ObjectId to string for JSON serialization
    for result in results:
        result['_id'] = str(result['_id'])

    return jsonify(results)


In [69]:
base_dir = os.path.dirname(os.path.abspath(__file__))
# Serve HTML file
@app.route('/')
def serve_html():
    return send_from_directory(base_dir, 'index.html')

# Serve JavaScript file
@app.route('/<path:path>')
def serve_file(path):
    return send_from_directory(base_dir, path)

In [73]:
# webbrowser.open('http://localhost:5000/')
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-10-22 02:54:46,399] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask_cors\extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\AdamKarner\anaconda3\Lib\site-packages\flask